Implemented: Inverse computation with regularization
- Uses np.linalg.solve for a and a_bar
- Uses reg instead of c for the regularization parameter

Not implemented: With Exceptions (yet)


# Verify formula with regularization 

[x] DONE 

$ y = (k + \frac{1}{c} I ) a $

$ y = (\tilde{K} + \tilde{c}^{-1} I ) a $

$ \frac{1}{\delta_s} (\tilde{K} + \tilde{c}^{-1} I )^{-1} 1 = \bar{a} $

Steps: 
* Set regularization
* Change k_create to include k_o
    * Set $\tilde{c}$ and $k_s$ and $k_d = 0$
* Look up and set $\tilde{K}$

In [2]:
import numpy as np
#list important:
# k_create,
# one_hot_sim, k_exp_horiz

#Set Parameters
n = 7
k_o = 0.4
k_s = 1.8
k_d = 0
delta_s = k_s - k_d
delta_o = k_o - k_d
alpha = 1 - (2 * (delta_o/delta_s))
alpha_prime = delta_o/delta_s
p = 5
q = 2
c_var = 0
lamb_val = np.arccosh((1+c_var)/(1-alpha))

K = {}
K_exp = {}
A = {}

arr_len = n-1
dual_len = (n-1) * 2
dual_len_exp = (n-1)*2 + 2
a = np.zeros(dual_len)
a_exp = np.zeros(dual_len_exp)
b = np.zeros(arr_len)
b_bar = np.zeros(arr_len)
c = np.zeros(1)
c_bar = np.zeros(1) 

reg = 2

def printer(var):
    print('{var}:', var)

def check_variables():
    print('n: ', n)
    print('alpha: ', alpha)
    print('k_s: ', k_s)
    print('k_o: ', k_o)
    print('alpha_prime: ', alpha_prime)
    print('c_var', c_var)
    print('lamb_val', lamb_val)
    print('a', a)
    print('a_exp', a_exp)
    print('b', b)
    print('c', c)


In [3]:
#K matrix without exceptions
    # return_k = false -> returns offset diagonal  
def tridiag(n):
    arr_len = n-1
    diag_len = arr_len-1
    B = np.diag(np.ones((diag_len)), k=1) + np.diag(np.ones((diag_len)), k=-1)
    return B        

def k_create(n, alpha_prime):
    arr_len = n-1
    A = k_s * np.identity(arr_len)
    D = A
    B = k_o*tridiag(n)
    C = B
    E = np.block([[A,B],[B,A]])
    return E

def k_tilde_create(n, alpha_prime):
    arr_len = n-1
    A = np.identity(arr_len)
    B = -alpha_prime*tridiag(n)
    E = A + B
    return E

def y_create(n):
    A = np.ones(n-1)
    B = A * -1
    D = np.concatenate([A, B])
    return D

def a_create(n, alpha_prime):
    K = k_create(n, alpha_prime)
    y = y_create(n)
    a = np.linalg.solve(K, y)
    return a

def a_create_reg(n, alpha_prime, c):
    K = k_create(n, alpha_prime)
    y = y_create(n)
    a = np.linalg.solve(K + (1/c)*np.identity((n-1)*2), y)
    return a


In [4]:
K = k_create(n,alpha_prime)
print('K', '\n', K)
print('K + (1/c)*np.identity((n-1)*2)', '\n', K + (1/reg)*np.identity((n-1)*2))
print('det:', np.linalg.det(K))
y = y_create(n)
a = a_create_reg(n, alpha_prime, reg)
printer(a)

K_tilde = k_tilde_create(n,alpha_prime)
a_bar = a[:len(a)//2]
printer(a_bar)

K 
 [[1.8 0.  0.  0.  0.  0.  0.  0.4 0.  0.  0.  0. ]
 [0.  1.8 0.  0.  0.  0.  0.4 0.  0.4 0.  0.  0. ]
 [0.  0.  1.8 0.  0.  0.  0.  0.4 0.  0.4 0.  0. ]
 [0.  0.  0.  1.8 0.  0.  0.  0.  0.4 0.  0.4 0. ]
 [0.  0.  0.  0.  1.8 0.  0.  0.  0.  0.4 0.  0.4]
 [0.  0.  0.  0.  0.  1.8 0.  0.  0.  0.  0.4 0. ]
 [0.  0.4 0.  0.  0.  0.  1.8 0.  0.  0.  0.  0. ]
 [0.4 0.  0.4 0.  0.  0.  0.  1.8 0.  0.  0.  0. ]
 [0.  0.4 0.  0.4 0.  0.  0.  0.  1.8 0.  0.  0. ]
 [0.  0.  0.4 0.  0.4 0.  0.  0.  0.  1.8 0.  0. ]
 [0.  0.  0.  0.4 0.  0.4 0.  0.  0.  0.  1.8 0. ]
 [0.  0.  0.  0.  0.4 0.  0.  0.  0.  0.  0.  1.8]]
K + (1/c)*np.identity((n-1)*2) 
 [[2.3 0.  0.  0.  0.  0.  0.  0.4 0.  0.  0.  0. ]
 [0.  2.3 0.  0.  0.  0.  0.4 0.  0.4 0.  0.  0. ]
 [0.  0.  2.3 0.  0.  0.  0.  0.4 0.  0.4 0.  0. ]
 [0.  0.  0.  2.3 0.  0.  0.  0.  0.4 0.  0.4 0. ]
 [0.  0.  0.  0.  2.3 0.  0.  0.  0.  0.4 0.  0.4]
 [0.  0.  0.  0.  0.  2.3 0.  0.  0.  0.  0.4 0. ]
 [0.  0.4 0.  0.  0.  0.  2.3 0.  0.  0.  0.

In [5]:
print((K + (1/reg)*np.identity((n-1)*2)) @ a)

[ 1.  1.  1.  1.  1.  1. -1. -1. -1. -1. -1. -1.]


In [6]:
def test_Ka_y_reg():
    assert np.allclose( (K + (1/reg)*np.identity((n-1)*2)) @ a, y), "if not equal"
    print('test: K@a = y', '\n')

test_Ka_y_reg()

test: K@a = y 



In [7]:
print(delta_s * K_tilde)
print(K)


[[ 1.8 -0.4  0.   0.   0.   0. ]
 [-0.4  1.8 -0.4  0.   0.   0. ]
 [ 0.  -0.4  1.8 -0.4  0.   0. ]
 [ 0.   0.  -0.4  1.8 -0.4  0. ]
 [ 0.   0.   0.  -0.4  1.8 -0.4]
 [ 0.   0.   0.   0.  -0.4  1.8]]
[[1.8 0.  0.  0.  0.  0.  0.  0.4 0.  0.  0.  0. ]
 [0.  1.8 0.  0.  0.  0.  0.4 0.  0.4 0.  0.  0. ]
 [0.  0.  1.8 0.  0.  0.  0.  0.4 0.  0.4 0.  0. ]
 [0.  0.  0.  1.8 0.  0.  0.  0.  0.4 0.  0.4 0. ]
 [0.  0.  0.  0.  1.8 0.  0.  0.  0.  0.4 0.  0.4]
 [0.  0.  0.  0.  0.  1.8 0.  0.  0.  0.  0.4 0. ]
 [0.  0.4 0.  0.  0.  0.  1.8 0.  0.  0.  0.  0. ]
 [0.4 0.  0.4 0.  0.  0.  0.  1.8 0.  0.  0.  0. ]
 [0.  0.4 0.  0.4 0.  0.  0.  0.  1.8 0.  0.  0. ]
 [0.  0.  0.4 0.  0.4 0.  0.  0.  0.  1.8 0.  0. ]
 [0.  0.  0.  0.4 0.  0.4 0.  0.  0.  0.  1.8 0. ]
 [0.  0.  0.  0.  0.4 0.  0.  0.  0.  0.  0.  1.8]]


In [ ]:
(K + (1/reg)*np.identity((n-1)*2)) @ a
reg_tilde = reg * delta_s

printer( ( (delta_s) * (K_tilde + (1/reg_tilde)*np.identity(len(K_tilde))) ) @ a_bar )

printer(a_bar)

printer( ( (1/delta_s) * np.linalg.inv(K_tilde + (1/reg_tilde)*np.identity(len(K_tilde))) ) @ np.ones(n-1) )

{var}: [1. 1. 1. 1. 1. 1.]
{var}: [0.54696663 0.64505811 0.6621175  0.6621175  0.64505811 0.54696663]
{var}: [0.54696663 0.64505811 0.6621175  0.6621175  0.64505811 0.54696663]


In [ ]:
reg_tilde = reg * delta_s
printer(reg_tilde)

{var}: 3.6
